In [18]:
import torch 
import numpy as np
import ipytest
ipytest.autoconfig()

## функция Convolution Transpose

In [19]:
def generate_bias(out_channels, bias):
    if bias:
        return torch.rand(out_channels)
    else:
        return torch.zeros(out_channels)

def generate_filter(in_channels, out_channels, kernel_size):
    if type(kernel_size) == tuple:
        return torch.rand(in_channels, out_channels, kernel_size[0], kernel_size[1])
    elif type(kernel_size) == int:
        return torch.rand(in_channels, out_channels, kernel_size, kernel_size)

def apply_dilation(total_mult, dilation):
    zero_tensor = torch.zeros(
        (total_mult.shape[0] - 1) * dilation + 1,
        (total_mult.shape[1] - 1) * dilation + 1
    )

    for a in range(0, zero_tensor.shape[0], dilation):
        for b in range(0, zero_tensor.shape[1], dilation):
            zero_tensor[a][b] = total_mult[a // dilation][b // dilation]

    return zero_tensor

def update_feature_map(feature_map, res, i, j, stride, dilation, filter):
    updated_map = np.add(
        feature_map[i * stride:i * stride + (filter.shape[2] - 1) * dilation + 1,
        j * stride:j * stride + (filter.shape[3] - 1) * dilation + 1],
        res
    )
    feature_map[i * stride:i * stride + (filter.shape[2] - 1) * dilation + 1,
    j * stride:j * stride + (filter.shape[3] - 1) * dilation + 1] = updated_map
    return feature_map

def process_padding(convolution, output_padding, padding):
    for t in range(len(convolution)):
        if output_padding > 0:
            padded = torch.nn.ConstantPad1d((0, output_padding, 0, output_padding), 0)
            convolution[t] = padded(convolution[t])

        convolution[t] = convolution[t][0 + padding:convolution[t].shape[0] - padding,
                         0 + padding:convolution[t].shape[1] - padding]

    return convolution

def ConvTranspose2d(in_channels, out_channels, kernel_size, stride=1, padding=0, output_padding=0, dilation=1, bias=True, padding_mode='zeros'):
    def convolution(matrix):
        if padding_mode != 'zeros':
            raise Exception('Only "zeros" padding mode is supported in ConvTranspose2d')

        bias_values = generate_bias(out_channels, bias)
        filter = generate_filter(in_channels, out_channels, kernel_size)

        convolution = []

        for l in range(out_channels):
            feature_map = torch.zeros(
                (matrix.shape[1] - 1) * stride + dilation * (kernel_size - 1) + 1,
                (matrix.shape[2] - 1) * stride + dilation * (kernel_size - 1) + 1
            )

            for c in range(in_channels):
                for i in range(0, matrix.shape[1]):
                    for j in range(0, matrix.shape[2]):
                        val = matrix[c][i][j]
                        total_mult = val * filter[c][l]
                        zero_tensor = apply_dilation(total_mult, dilation)
                        feature_map = update_feature_map(feature_map, zero_tensor, i, j, stride, dilation, filter)

            convolution.append(np.add(feature_map, np.full((feature_map.shape), bias_values[l])))

        convolution = process_padding(convolution, output_padding, padding)

        return convolution, filter, torch.tensor(bias_values)

    return convolution

In [20]:
def test_1():
    tensor2 = torch.rand(3, 18, 18)
    myFunction2 = ConvTranspose2d(in_channels=3, out_channels=2, kernel_size=3, stride=10, padding=0,output_padding=0, dilation=3, bias=True, padding_mode='zeros')
    result, kernel, bias_val = myFunction2(tensor2)
    torchFunction2 = torch.nn.ConvTranspose2d(in_channels=3, out_channels=2, kernel_size=3, stride=10, padding=0,output_padding=0, dilation=3, bias=True, padding_mode='zeros')
    torchFunction2.weight.data = kernel
    torchFunction2.bias.data = bias_val
    result_array = np.array(result) if isinstance(result, list) else result.numpy()
    assert np.allclose(result_array, torchFunction2(tensor2).data.numpy())

def test_2():
    tensor2 = torch.rand(1, 11, 11)
    myFunction2 = ConvTranspose2d(in_channels=1, out_channels=1, kernel_size=1, stride=1, padding=0,output_padding=0, dilation=3, bias=True, padding_mode='zeros')
    result, kernel, bias_val = myFunction2(tensor2)
    torchFunction2 = torch.nn.ConvTranspose2d(in_channels=1, out_channels=1, kernel_size=1, stride=1, padding=0,output_padding=0, dilation=3, bias=True, padding_mode='zeros')
    torchFunction2.weight.data = kernel
    torchFunction2.bias.data = bias_val
    result_array = np.array(result) if isinstance(result, list) else result.numpy()
    assert np.allclose(result_array, torchFunction2(tensor2).data.numpy())

def test_3():
    tensor2 = torch.rand(3, 10, 10)
    myFunction2 = ConvTranspose2d(in_channels=3, out_channels=2, kernel_size=3, stride=10, padding=0,output_padding=0, dilation=3, bias=True, padding_mode='zeros')
    result, kernel, bias_val = myFunction2(tensor2)
    torchFunction2 = torch.nn.ConvTranspose2d(in_channels=3, out_channels=2, kernel_size=3, stride=10, padding=0,output_padding=0, dilation=3, bias=True, padding_mode='zeros')
    torchFunction2.weight.data = kernel
    torchFunction2.bias.data = bias_val
    result_array = np.array(result) if isinstance(result, list) else result.numpy()
    assert np.allclose(result_array, torchFunction2(tensor2).data.numpy())

def test_4():
    tensor2 = torch.rand(2, 12, 12)
    myFunction2 = ConvTranspose2d(in_channels=2, out_channels=2, kernel_size=3, stride=10, padding=0,output_padding=0, dilation=3, bias=True, padding_mode='zeros')
    result, kernel, bias_val = myFunction2(tensor2)
    torchFunction2 = torch.nn.ConvTranspose2d(in_channels=2, out_channels=2, kernel_size=3, stride=10, padding=0,output_padding=0, dilation=3, bias=True, padding_mode='zeros')
    torchFunction2.weight.data = kernel
    torchFunction2.bias.data = bias_val
    result_array = np.array(result) if isinstance(result, list) else result.numpy()
    assert np.allclose(result_array, torchFunction2(tensor2).data.numpy())

## Доп. Задание: реализовать алгоритм работы транспонированной свертки, через алгоритм двумерной свертки, реализованный в первой лабораторной.

In [21]:
def myTranspConv2dInverse(in_channels, out_channels, kernel_size, transp_stride=1, padding=0, dilation=1, bias=True, padding_mode='zeros'):
    def convolution(matrix):
        # Рассчитываем паддинг для добавления к входной матрице
        pad = kernel_size - 1
        result_matrix = []

        # Применяем транспонирование с учетом заданного шага (transp_stride)
        for matr in matrix:
            zero_tensor = np.zeros((((matr.shape[0] - 1) * transp_stride) + 1, ((matr.shape[1] - 1) * transp_stride) + 1))
            for a in range(0, zero_tensor.shape[0], transp_stride):
                for b in range(0, zero_tensor.shape[1], transp_stride):
                    zero_tensor[a][b] = matr[a // (transp_stride)][b // (transp_stride)]

            # Добавляем паддинг к полученной матрице
            pad_matr = np.pad(zero_tensor, pad_width=pad, mode='constant')
            result_matrix.append(pad_matr)

        # Преобразуем входную матрицу в тензор PyTorch
        matrix = torch.tensor(result_matrix)

        # Генерация смещения 
        if bias:
            # Если включено смещение, создаем случайные значения для смещения
            bias_values = torch.rand(out_channels)
        else:
            # Иначе устанавливаем смещение в нули
            bias_values = torch.zeros(out_channels)

        # Применяем режим заполнения в зависимости от заданного значения (padding_mode)
        if padding_mode == 'zeros':
            pad = torch.nn.ZeroPad2d(padding)
            matrix = pad(matrix)
        elif padding_mode == 'reflect':
            pad = torch.nn.ReflectionPad2d(padding)
            matrix = pad(matrix)
        elif padding_mode == 'replicate':
            pad = torch.nn.ReplicationPad2d(padding)
            matrix = pad(matrix)
        elif padding_mode == 'circular':
            pass

        # Генерация случайного ядра свертки
        filter = np.array(torch.rand(out_channels, in_channels, kernel_size, kernel_size))

        # Инвертирование ядра для ConvTranspose2d
        filter_for_transpose = []

        # Внешний цикл по количеству выходных каналов (out_channels)
        for j in range(out_channels):
            # Инициализация временного списка filter_in для текущего выходного канала
            filter_in = []

            # Внутренний цикл по количеству входных каналов (in_channels)
            for i in range(in_channels):
                # Добавление транспонированного фильтра в filter_in.
                # np.flip() - выполнение операции "flip" меняетсяя порядок элементов массива по заданной оси.
                filter_in.append(np.flip(np.array(filter[j][i])))

            # Добавление filter_in (транспонированного фильтра для текущего выходного канала) в filter_for_transpose
            filter_for_transpose.append(filter_in)

        # Преобразование списка filter_for_transpose в тензор torch
        filter_for_transpose = torch.tensor(filter_for_transpose)

        # Изменение формы тензора filter_for_transpose в необходимую форму с учетом размеров ядра (kernel_size)
        filter_for_transpose = filter_for_transpose.reshape(in_channels, out_channels, kernel_size, kernel_size)

        # Начальные значения для шага и результата свертки
        stride = 1
        convolution = []

        # Проходим по выходным каналам (out_channels)
        for l in range(out_channels):
            feature_map = np.array([])  # Генерация пустой feature-map

            # Проходим по высоте и ширине матрицы с учетом размера ядра и диляции
            for i in range(0, matrix.shape[1] - ((filter.shape[2] - 1) * dilation + 1) + 1, stride):
                for j in range(0, matrix.shape[2] - ((filter.shape[3] - 1) * dilation + 1) + 1, stride):
                    total = 0

                    # Проходим по входным каналам (in_channels)
                    for c in range(in_channels):

                        val = matrix[c][i:i + (filter.shape[2] - 1) * dilation + 1:dilation,
                              j:j + (filter.shape[3] - 1) * dilation + 1:dilation]
                        
                        local_sum = (val * filter[l][c]).sum()
                        total = total + local_sum

                    # Добавляем результат свертки и смещение к feature-map
                    feature_map = np.append(feature_map, float(total + bias_values[l]))

            # Преобразуем feature-map в матрицу и добавляем к результату свертки
            convolution.append(feature_map.reshape(
                (matrix.shape[1] - ((filter.shape[2] - 1) * dilation + 1)) // stride + 1,
                (matrix.shape[2] - ((filter.shape[3] - 1) * dilation + 1)) // stride + 1))

        return np.array(convolution), torch.tensor(np.array(filter_for_transpose)), torch.tensor(np.array(bias_values))

    return convolution

In [22]:
def run_test2(test_name, in_channels, out_channels, kernel_size, transp_stride, input_shape):
    tensor = torch.rand(*input_shape)
    my_function = myTranspConv2dInverse(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, transp_stride=transp_stride, bias=True)
    result, kernel, bias_val = my_function(tensor)
    
    torch_function = torch.nn.ConvTranspose2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=transp_stride, bias=True)
    torch_function.weight.data = kernel
    torch_function.bias.data = bias_val
    
    my_result = str(np.round(result, 2))
    torch_result = str(np.round(np.array(torch_function(tensor).data), 2))
    assert torch_result == my_result, f"Test {test_name} failed"

In [23]:
def test_5():
    run_test2("test_1", in_channels=3, out_channels=1, kernel_size=3, transp_stride=2, input_shape=(3, 5, 6))

def test_6():
    run_test2("test_2", in_channels=1, out_channels=2, kernel_size=4, transp_stride=3, input_shape=(1, 28, 28))

def test_7():
    run_test2("test_3", in_channels=7, out_channels=1, kernel_size=3, transp_stride=5, input_shape=(7, 10, 10))

def test_8():
    run_test2("test_3", in_channels=7, out_channels=1, kernel_size=3, transp_stride=5, input_shape=(7, 10, 10))

In [24]:
ipytest.run()

........                                                                                     [100%]
======================================== warnings summary =========================================
t_7fef9cbb342147b98053bdf03afc06b1.py::test_1
t_7fef9cbb342147b98053bdf03afc06b1.py::test_2
t_7fef9cbb342147b98053bdf03afc06b1.py::test_3
t_7fef9cbb342147b98053bdf03afc06b1.py::test_4
  C:\Users\Никола\AppData\Local\Temp\ipykernel_12176\3863638600.py:74: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
    return convolution, filter, torch.tensor(bias_values)

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
8 passed, 4 warnings in 5.66s


<ExitCode.OK: 0>